In [1]:
# ==========================================
# 1: INSTALLATION & SETUP
# ==========================================
!pip install -qU langchain==0.1.0 langchain-google-genai==0.0.9 langchain-community==0.0.19 python-dotenv

In [2]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=77da87c9db7bce06945d84e2542f31edb5e45b6569a82a069fea8c0d73f18211
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [3]:
import os
from google.colab import userdata
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SerpAPIWrapper

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
# ==========================================
# 2. CONFIGURATION
#🔑 SETUP API KEY
# ==========================================
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1,convert_system_message_to_human=True)

os.environ["SERPAPI_API_KEY"] = "YOUR_API_KEY"

print("✅ Libraries installed and LLM ready.")

✅ Libraries installed and LLM ready.


In [6]:
# ==========================================
# 3. DEFINE THE TOOLS (THE WORKERS)
# ==========================================
def news_search(query):

    """Worker 1: The News Reporter.
    Strictly fails if asked for numbers/prices."""

    # --- LOGIC: The "Self-Realization" of Failure ---
    # If the query is about math/money, this tool intentionally crashes
    # to trigger the hand-off.
    forbidden_keywords = ["price", "trading", "ratio", "value", "number"]
    if any(word in query.lower() for word in forbidden_keywords):
        raise ValueError("I am a News Reporter! I cannot handle specific numbers or stock prices.")

    # Otherwise, perform a news search
    search = SerpAPIWrapper(params={"tbm": "nws"}) # tbm=nws forces Google News results
    return search.run(f"{query} latest headlines")

def stock_search(query):
    """Worker 2: The Stock Broker.
    Finds specific market data."""
    #search = SerpAPIWrapper(params={"engine": "google_finance"})
    # Or standard search if finance engine not available in your plan:
    search = SerpAPIWrapper(params={"engine": "google"})
    #return search.run(f"{query} stock price") --->This code is also running

    #Following logic is to solve Identity Crisis problem -
    #This forces the LLM to trust the price even if the name looks different.
    #(e.g., Google $\to$ Alphabet, Facebook $\to$ Meta, Zomato $\to$ Eternal).

    result = search.run(f"{query} stock price")

    # <--- THE UNIVERSAL FIX --->
    # We append a GENERIC instruction to the result.
    # This tells the Agent: "If you see a price, trust it, even if the name is weird."
    return (
        f"Search Result: {result}\n"
        f"SYSTEM NOTE: The company name in the search result might differ from the query "
        f"(e.g., Parent Company Name or Legal Entity Name). "
        f"If a stock price is visible in the result, assume it is the correct data for '{query}' and return it."
    )

# Wrap them as langchain tools!
news_tool = Tool(
    name="NewsSearch",
    func=news_search,
    description="Useful ONLY for finding text articles and headlines. Fails on numbers/prices."
)

stock_tool = Tool(
    name="StockSearch",
    func=stock_search,
    description="Useful for finding current stock prices, market ratios, and numeric data."
)

In [7]:
# ==========================================
# 4. DEFINE THE AGENTS
# ==========================================

# Create Individual Agents ---
# We use ZERO_SHOT_REACT_DESCRIPTION because it handles simple string inputs better.

reporter_agent = initialize_agent(
    tools=[news_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True # Helps agent recover if it messes up formatting
)

stock_broker_agent = initialize_agent(
    tools=[stock_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
# =========================================================
# 5. THE ROUTER (THE SUPERVISOR) - Hybrid Archiecture
# =========================================================
# This small chain decides which agent to call first.

router_template = """
You are a Supervisor. Classify the user's intent into exactly one of two categories: "NEWS" or "STOCK".

Query: {query}

If the user wants current events, headlines, or text news: return "NEWS"
If the user wants numbers, prices, ratios, or market data: return "STOCK"

Category:"""

router_prompt = PromptTemplate(template=router_template, input_variables=["query"])
router_chain = LLMChain(llm=llm, prompt=router_prompt)


In [13]:
# ==========================================
# 6. THE ORCHESTRATOR (ROUTER + FALLBACK)
# ==========================================
def run_smart_swarm(query):
    print(f"\n🔹 User Query: '{query}'")

    # --- STEP 1: ROUTING (The Brain) ---
    print("🧠 Supervisor: Deciding who should handle this...")
    category = router_chain.run(query).strip().upper()
    print(f"👉 Supervisor assigned task to: [{category} AGENT]")

    # Define primary and fallback agents based on decision
    if "STOCK" in category:
        primary_agent = stock_broker_agent
        primary_name = "Stock Broker"
        fallback_agent = reporter_agent
        fallback_name = "News Reporter"
    else:
        primary_agent = reporter_agent
        primary_name = "News Reporter"
        fallback_agent = stock_broker_agent
        fallback_name = "Stock Broker"

    # --- STEP 2: EXECUTION WITH FALLBACK (The Safety Net) ---
    try:
        response = primary_agent.run(query)
        return response

    except Exception as e:
        print(f"\n❌ {primary_name} CRASHED: {e}")
        print(f"🔄 SYSTEM ALERT: Triggering Fallback to [{fallback_name}]...\n")

        # Retry with the other agent
        return fallback_agent.run(query)

In [14]:
# ==========================================
# 7. RUN DEMO
# ==========================================
if __name__ == "__main__":
    print("\n🤖 SWARM INTELLIGENCE: Router + Fallback Enabled.\n")

    # Test 1: Clear Stock Question (Router should pick Stock, Success)
    print("--- TEST 1: STOCK QUESTION ---")
    print(run_smart_swarm("What is Zomato trading at?"))

    # Test 2: Clear News Question (Router should pick News, Success)
    print("\n--- TEST 2: NEWS QUESTION ---")
    print(run_smart_swarm("What is the latest news about Tata Motors?"))

    # Test 3: Tricky Question (Router might pick News, but it requires numbers -> Hand-off saves it)
    print("\n--- TEST 3: TRICKY / AMBIGUOUS QUESTION ---")
    # Asking for "News on price" might trick the Router into picking "NEWS",
    # causing the News Agent to crash, and demonstrating the Fallback.
    print(run_smart_swarm("Give me a news article about the exact price of Reliance."))


🤖 SWARM INTELLIGENCE: Router + Fallback Enabled.

--- TEST 1: STOCK QUESTION ---

🔹 User Query: 'What is Zomato trading at?'
🧠 Supervisor: Deciding who should handle this...
👉 Supervisor assigned task to: [STOCK AGENT]


> Entering new AgentExecutor chain...
Action: StockSearch
Action Input: Zomato
Observation: Search Result: {'type': 'finance_results', 'title': 'News', 'exchange': 'NSE', 'stock': 'ETERNAL', 'currency': 'INR', 'price': 285.2}
SYSTEM NOTE: The company name in the search result might differ from the query (e.g., Parent Company Name or Legal Entity Name). If a stock price is visible in the result, assume it is the correct data for 'Zomato' and return it.
Thought:I now know the final answer
Final Answer: Zomato is trading at 285.2 INR.

> Finished chain.
Zomato is trading at 285.2 INR.

--- TEST 2: NEWS QUESTION ---

🔹 User Query: 'What is the latest news about Tata Motors?'
🧠 Supervisor: Deciding who should handle this...
👉 Supervisor assigned task to: [NEWS AGENT]


> E